In [1]:
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
import sys
# np.set_printoptions(threshold=sys.maxsize)

In [3]:
nvda = yf.Ticker('NVDA')

nvda_hist = nvda.history(period='5y', interval='1d', end='2025-06-02')
nvda_hist.drop(columns=['Dividends', 'Stock Splits'], inplace=True)
dates = pd.to_numeric(nvda_hist.index)
dates = dates.to_numpy()

print(nvda_hist)

                                 Open        High         Low       Close  \
Date                                                                        
2020-06-29 00:00:00-04:00    9.139624    9.174260    8.870759    9.169774   
2020-06-30 00:00:00-04:00    9.283398    9.494950    9.236054    9.466544   
2020-07-01 00:00:00-04:00    9.489471    9.544291    9.382075    9.498691   
2020-07-02 00:00:00-04:00    9.607332    9.705509    9.546782    9.580670   
2020-07-06 00:00:00-04:00    9.709746    9.876197    9.680592    9.806926   
...                               ...         ...         ...         ...   
2025-06-20 00:00:00-04:00  145.449997  146.199997  142.649994  143.850006   
2025-06-23 00:00:00-04:00  142.500000  144.779999  142.029999  144.169998   
2025-06-24 00:00:00-04:00  145.559998  147.960007  145.500000  147.899994   
2025-06-25 00:00:00-04:00  149.270004  154.449997  149.259995  154.309998   
2025-06-26 00:00:00-04:00  155.979996  156.720001  154.000000  155.020004   

In [4]:
from sklearn.preprocessing import StandardScaler, Normalizer

scaler = StandardScaler()
dates_scaled = scaler.fit_transform(dates.reshape(-1, 1))


dates_norm = dates / max(dates)
for column in nvda_hist.columns:
    nvda_hist[column]/=max(nvda_hist[column])
nvda_hist.head(-5)

,Open,High,Low,Close,Volume
Date,,,,,
2020-06-29 00:00:00-04:00,0.058595,0.058539,0.057602,0.059152,0.221676
2020-06-30 00:00:00-04:00,0.059517,0.060585,0.059974,0.061067,0.238286
2020-07-01 00:00:00-04:00,0.060838,0.060900,0.060923,0.061274,0.211572
2020-07-02 00:00:00-04:00,0.061593,0.061929,0.061992,0.061803,0.235801
2020-07-06 00:00:00-04:00,0.062250,0.063018,0.062861,0.063262,0.204605
...,...,...,...,...,...
2025-06-12 00:00:00-04:00,0.910181,0.925217,0.921104,0.935363,0.105165
2025-06-13 00:00:00-04:00,0.913450,0.916156,0.914610,0.915817,0.117119
2025-06-16 00:00:00-04:00,0.919028,0.932746,0.929870,0.933363,0.118617


In [ ]:
class CNNModel(nn.Module):
    def __init__(self, input_features, num_features, hidden_units, out_features):
        self.l1 = nn.Sequential(
            nn.Conv1d(in_channels=input_features, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(num_features=num_features),
            nn.ReLU()
        )
        self.l2 = nn.Sequential(
            [
                nn.Sequential(
                    nn.Conv1d(in_channels=hidden_units, out_channels=out_features, kernel_size=3, stride=1, padding=1),
                    nn.BatchNorm1d(num_features=num_features),
                    nn.ReLU()
                )
            ] for _ in range(1, 10)
        )

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        return x